In [1]:
from mip import Model, xsum, maximize, BINARY, OptimizationStatus

In [2]:
import pandas as pd
from random import randint, randrange

случайным образом выбираем размер массива N в диапазоне (200, 1000)

In [3]:
n = randint(200, 1000) 
N = range(n)
print("n: ", n)

n:  225


случайным образом выбираем четный размер подмн-ва S мн-ва N в диапазоне (2, n)

In [4]:
s = randrange(2, n, 2)
S = range(s)
print("s: ", s)

s:  24


Ns = N\S (разность мн-ва N и S) 

In [5]:
Ns = [j for j in N if j >= len(S)] 

случайным образом создаем массивы cj и aj в диапозоне (1, 50) и (1, 30) соответственно

In [6]:
c = [randint(1, 50) for i in N]
a = [randint(1, 30) for i in N]
c_a = [c[i]/a[i] for i in N]
df = pd.DataFrame({'values': c, 'weights': a, 'values/weights': c_a})
df.sort_values('values/weights', ascending=False).head(10)

,values,weights,values/weights
218,42,1,42.0
113,35,1,35.0
33,30,1,30.0
67,29,1,29.0
61,27,1,27.0
194,25,1,25.0
83,49,2,24.5
212,40,2,20.0
154,35,2,17.5
48,34,2,17.0


случайным образом создаем ограничение b в диапозоне (1, 100)

In [7]:
b = randint(20, 100)
print("b: ", b)

b:  88


# создаем модель

In [8]:
m = Model("knapsack")

# элементы решения xj = {0, 1}
x = [m.add_var(var_type=BINARY) for j in N]

# целевая функция
m.objective = maximize(xsum(c[j] * x[j] for j in N))

# добовляем ограничения
m += xsum(a[j] * x[j] for j in N) <= b

m.add_lazy_constr(xsum(x[j] for j in S) - xsum(x[j] for j in Ns) <= len(S) - 1)

status = m.optimize(max_seconds=300)

print("status Optimal: ", status == OptimizationStatus.OPTIMAL)

status Optimal:  True


## Результат

In [9]:
def usefullness(t):
        return t[1] / t[2]
    
def exchange_item(items, total_items):
    
    items_big_size = []
    for item in items:
        if item[2] > 1:
            items_big_size.append(item)
            
    def change(items_big_size, total_items):
        
        total_items_new = []
        
        for item in items_big_size:
            for t_item in total_items:
                if item[0] != t_item[0]:
                    if item[2] > t_item[2]:
                        total_items_new.append(t_item)
        
        n = len(total_items_new)
        items_new_odd = {}
        
        for item in items_big_size:
            b = item[2]
            
            model = Model("knapsack")

            x = [model.add_var(var_type=BINARY) for i in range(n)]
            model.objective = maximize(xsum(total_items_new[i][1] * x[i] for i in range(n)))
            model += xsum(total_items_new[i][2] * x[i] for i in range(n)) <= b
            model.optimize(max_seconds=300)
            selected = [i for i in range(n) if x[i].x >= 0.99]
            if len(selected) % 2 == 0:
                items_new_odd[item[0]] = [(i, total_items_new[i][1], total_items_new[i][2]) for i in selected]
        
        if len(items_new_odd) == 0:
            print("No optimal possibility of odd items")
            return items
        
        max_c = 0
        for item_odd in items_new_odd:
            sum_c = sum([i[1] for i in items_new_odd[item_odd]])
            if sum_c > max_c:
                max_c = sum_c
                items_new = items_new_odd[item_odd]
                index = item_odd
        
        return items_new, index
    
    if len(items_big_size) == 0:  
        items.sort(key=usefullness, reverse=True) 
        return items, items[-1][0]
    
    else:
        return change(items_big_size, total_items)   

In [10]:
    
selected = [j for j in N if x[j].x >= 0.99]
selected.sort()

count_items = len(selected)

items = [(i, c[i], a[i]) for i in selected]
items.sort(key=usefullness, reverse=True) 

total_items = [(i, c[i], a[i]) for i in N]


In [11]:
print("len(N) = ", n)
print("len(S) = ", s)
print("b = ", b)
print("_____________")

    
if count_items % 2 == 0:
    print("even: ", count_items)
    print("try even -> odd ")
    
    result = exchange_item(items, total_items)
    
    print("replase ", (result[1], c[result[1]], a[result[1]]), "on", result[0])
    items_new = result[0]
    index_pop = result[1]
    for i in range(len(items)):
        if items[i][0] == index_pop:
            items.pop(i)
            break

    items = items + items_new

    print("items count: ", len(items)) 
    size = 0
    value = 0
    for t in items:
        size = size + t[2]
        value = value + t[1]

    items.sort(key=usefullness, reverse=True)  

    print("value: ", value, ", size: ", size, ", capacities: ", b)
    print("_____________")
    print(pd.DataFrame({'values': [t[1] for t in items], 'weights': [t[2] for t in items]}, index=[t[0] for t in items]))
    print("_____________")

else: 
    print("odd: ", count_items)
    print("selected items: {}".format(selected))
    print("_____________")
    print("items count: ", len(selected))

    size = 0
    value = 0
    for t in items:
        size = size + t[2]
        value = value + t[1]

    items.sort(key=usefullness, reverse=True)  

    print("value: ", value, ", size: ", size, ", capacities: ", b)
    print("_____________")
    print(pd.DataFrame({'values': [t[1] for t in items], 'weights': [t[2] for t in items]}, index=[t[0] for t in items]))
    print("_____________")

len(N) =  225
len(S) =  24
b =  88
_____________
even:  28
try even -> odd 
replase  (176, 49, 6) on [(15, 42, 1), (62, 42, 1), (70, 42, 1), (89, 42, 1), (242, 42, 1), (399, 42, 1)]
items count:  33
value:  1147 , size:  88 , capacities:  88
_____________
     values  weights
218      42        1
15       42        1
62       42        1
70       42        1
89       42        1
242      42        1
399      42        1
113      35        1
33       30        1
67       29        1
61       27        1
194      25        1
83       49        2
212      40        2
154      35        2
48       34        2
38       50        3
109      25        2
107      23        2
157       9        1
125      17        2
191      34        4
164      42        5
40       48        6
89       48        6
92       45        6
25       43        6
111      21        3
175      28        4
132      27        4
143      27        4
51       31        5
220      31        5
_____________


In [12]:
# сохранение модели
m.write('model_knapsack.lp')